In [45]:
import time
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')

In [46]:
conn_string = 'postgresql://x:x@casestudy.cpklq9vljodx.us-east-1.rds.amazonaws.com/casestudy'


In [47]:
df = pd.read_csv('https://betterai.s3.amazonaws.com/US_COVID_SHORT_SAMPLE+_+Data+Challenge.csv', date_parser=dateparse)

In [48]:
df

,submission_date,state,total_cases,new_case,total_deaths,new_death
0,1/22/2020,AK,0,0,0,0
1,1/23/2020,AK,0,0,0,0
2,1/24/2020,AK,0,0,0,0
3,1/25/2020,AK,0,0,0,0
4,1/26/2020,AK,0,0,0,0
...,...,...,...,...,...,...
29335,5/20/2021,WY,"59,590",214,713,0
29336,5/21/2021,WY,"59,660",70,713,0
29337,5/22/2021,WY,"59,705",45,713,0
29338,5/23/2021,WY,"59,750",45,713,0


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29340 entries, 0 to 29339
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   submission_date  29340 non-null  object
 1   state            29340 non-null  object
 2   total_cases      29340 non-null  object
 3   new_case         29340 non-null  object
 4   total_deaths     29340 non-null  object
 5   new_death        29340 non-null  object
dtypes: object(6)
memory usage: 1.3+ MB


In [50]:
df.describe()

,submission_date,state,total_cases,new_case,total_deaths,new_death
count,29340,29340,29340,29340,29340,29340
unique,489,60,21238,5117,9733,431
top,11/4/2020,AR,0,0,0,0
freq,60,489,3749,5964,5120,10141


In [51]:
df['total_cases']= df['total_cases'].str.replace(',','').astype(int)
df['total_deaths']= df['total_deaths'].str.replace(',','').astype(int)
df['new_case']= df['new_case'].str.replace(',','').astype(int)
df['new_death']= df['new_death'].str.replace(',','').astype(int)

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29340 entries, 0 to 29339
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   submission_date  29340 non-null  object
 1   state            29340 non-null  object
 2   total_cases      29340 non-null  int32 
 3   new_case         29340 non-null  int32 
 4   total_deaths     29340 non-null  int32 
 5   new_death        29340 non-null  int32 
dtypes: int32(4), object(2)
memory usage: 917.0+ KB


In [53]:
df.loc[df['new_case'] <= 20, 'covid_cases_rate'] = 'LOW'
df.loc[df['new_case'] > 50, 'covid_cases_rate'] = 'HIGH'
df.loc[(df['new_case']> 20) & (df['new_case']  <= 50),'covid_cases_rate'] = 'MEDIUM'

In [54]:
df.loc[df['new_death'] <= 5, 'covid_deaths_rate'] = 'LOW'
df.loc[df['new_death'] > 10, 'covid_deaths_rate'] = 'HIGH'
df.loc[(df['new_death']> 5) & (df['new_death']  <= 10),'covid_deaths_rate'] = 'MEDIUM'

In [55]:
df.info()
display(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29340 entries, 0 to 29339
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   submission_date    29340 non-null  object
 1   state              29340 non-null  object
 2   total_cases        29340 non-null  int32 
 3   new_case           29340 non-null  int32 
 4   total_deaths       29340 non-null  int32 
 5   new_death          29340 non-null  int32 
 6   covid_cases_rate   29340 non-null  object
 7   covid_deaths_rate  29340 non-null  object
dtypes: int32(4), object(4)
memory usage: 1.3+ MB


,submission_date,state,total_cases,new_case,total_deaths,new_death,covid_cases_rate,covid_deaths_rate
0,1/22/2020,AK,0,0,0,0,LOW,LOW
1,1/23/2020,AK,0,0,0,0,LOW,LOW
2,1/24/2020,AK,0,0,0,0,LOW,LOW
3,1/25/2020,AK,0,0,0,0,LOW,LOW
4,1/26/2020,AK,0,0,0,0,LOW,LOW
...,...,...,...,...,...,...,...,...
29335,5/20/2021,WY,59590,214,713,0,HIGH,LOW
29336,5/21/2021,WY,59660,70,713,0,HIGH,LOW
29337,5/22/2021,WY,59705,45,713,0,MEDIUM,LOW
29338,5/23/2021,WY,59750,45,713,0,MEDIUM,LOW


In [56]:
db = create_engine(conn_string)

In [57]:
conn = db.connect()

In [58]:
df.to_sql('casestudy_table', con=conn, if_exists='replace', index=False)

In [59]:
sql = '''SELECT submission_date, state, total_cases, new_case, total_deaths, new_death, covid_cases_rate, covid_deaths_rate,TO_DATE(submission_date, 'MM/DD/YYYY') t FROM public.casestudy_table'''
pd.read_sql(sql,conn)

,submission_date,state,total_cases,new_case,total_deaths,new_death,covid_cases_rate,covid_deaths_rate,t
0,1/22/2020,AK,0,0,0,0,LOW,LOW,2020-01-22
1,1/23/2020,AK,0,0,0,0,LOW,LOW,2020-01-23
2,1/24/2020,AK,0,0,0,0,LOW,LOW,2020-01-24
3,1/25/2020,AK,0,0,0,0,LOW,LOW,2020-01-25
4,1/26/2020,AK,0,0,0,0,LOW,LOW,2020-01-26
...,...,...,...,...,...,...,...,...,...
29335,5/20/2021,WY,59590,214,713,0,HIGH,LOW,2021-05-20
29336,5/21/2021,WY,59660,70,713,0,HIGH,LOW,2021-05-21
29337,5/22/2021,WY,59705,45,713,0,MEDIUM,LOW,2021-05-22
29338,5/23/2021,WY,59750,45,713,0,MEDIUM,LOW,2021-05-23


In [61]:
sql = '''SELECT 'Total cases and deaths in USA' Note ,sum(new_case) TotalCases, sum(new_death) TotalDeaths
FROM public.casestudy_table;'''
pd.read_sql(sql,conn)

,note,totalcases,totaldeaths
0,Total cases and deaths in USA,32892576,576202


In [62]:
sql = '''select  * from
(select state, sum(new_case) TotalCases
FROM public.casestudy_table
group by state) a
order by TotalCases desc
limit 5'''
pd.read_sql(sql,conn)

,state,totalcases
0,CA,3674044
1,TX,2850091
2,FL,2268729
3,IL,1376441
4,PA,1196562


In [63]:
sql = '''SELECT 'Total cases and deaths in California' Note ,sum(new_case) TotalCases, sum(new_death) TotalDeaths
FROM public.casestudy_table
where state = 'CA'
and TO_DATE(submission_date, 'MM/DD/YYYY') <= '5/20/2021';'''
pd.read_sql(sql,conn)

,note,totalcases,totaldeaths
0,Total cases and deaths in California,3673029,61540


In [42]:
sql = '''WITH tab AS (
SELECT submission_date , state, total_cases, new_case, (new_case/total_cases::float)*100 PercentNewCases
FROM public.casestudy_table
where state = 'CA' and total_cases > 0 
ORDER BY TO_DATE(submission_date, 'MM/DD/YYYY') asc
) ,
tab2 as (
SELECT
submission_date, 
PercentNewCases,
LAG(PercentNewCases,1) OVER (
ORDER BY submission_date
) PrevPercentNewCases
,PercentNewCases - 	LAG(PercentNewCases,1) OVER (
ORDER BY submission_date
) diff
FROM
tab
)
select * from tab2 where diff >= 25;'''
pd.read_sql(sql,conn)

,submission_date,percentnewcases,prevpercentnewcases,diff
0,1/26/2020,100.000000,0.618376,99.381624
1,1/31/2020,33.333333,0.303490,33.029843
2,2/1/2020,88.888889,0.190801,88.698088
3,2/7/2020,41.772152,0.194801,41.577351


In [43]:
sql = '''with tab as(
SELECT state ,sum(new_case) totalcases, sum(new_death) totaldeaths
FROM public.casestudy_table
group by state),
tab2 as(
select state, (totaldeaths/totalcases::float)*100 deathrate  from tab 
where totalcases > 0
)
select * from tab2
order by deathrate ;'''
pd.read_sql(sql,conn)

,state,deathrate
0,RMI,0.000000
1,FSM,0.000000
2,AK,0.538698
3,UT,0.566450
4,VI,0.798108
5,NE,1.008276
6,VT,1.056907
7,ID,1.089881
8,MP,1.104972
9,WI,1.152112
